#                  **Wellness Bot with Sentiment Analysis**
**Overview:-**

This project builds a chatbot that adapts its responses based on user sentiment, providing empathetic and personalized replies. It uses Hugging Face models for both sentiment analysis and text generation and is deployed with Gradio for user interaction.

**Key Features:**

**Sentiment Analysis:** DistilBERT identifies positive or negative sentiment in user input.

**Text Generation:** Llama-2 model generates conversational responses based on input and sentiment.

**Dynamic Responses:** The bot adjusts its tone, offering empathy for negative sentiment and encouragement for positive sentiment.\
**Interactive Interface:** Gradio provides a simple text-based interface for users to chat with the bot. The conversation history can be reset with the "clear" command.

**Purpose:-**
The chatbot is designed for applications like mental health support, customer service, and education by offering sentiment-aware, personalized conversations.

# **Step 1: Install Necessary Libraries**
Install Required Libraries
This step installs the necessary libraries including PyTorch, Hugging Face's transformers,Gradio (for building the user interface), and sentence-transformers for sentiment analysis.

In [ ]:
!pip install torch torchvision torchaudio transformers sentencepiece gradio sentence-transformers accelerate

# **Step 2: Import Required Libraries**
Importing PyTorch and Hugging Face modules for working with models and Gradio for creating the chatbot interface.

In [ ]:
import torch
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
import gradio as gr

# **Step 3:Sentiment Analysis Pipeline**
Initialize Sentiment Analysis Pipeline
Using a pre-trained **DistilBERT** model to perform sentiment analysis. This model is fine-tuned on the SST-2 dataset,
which is designed for sentiment classification tasks (positive or negative sentiment).

In [ ]:
sentiment_analyzer = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")


# **Step 4:Load Llama-2 Model and Tokenizer**
Load Llama Model for Text Generation
The **Llama-2-7b-chat-hf model** from Hugging Face is loaded to handle conversational text generation.
The model is loaded in half-precision (float16) for faster computation, and the device_map is set to "auto" to automatically
use available hardware (such as GPUs, if available)

In [ ]:
llama_model_id = "NousResearch/Llama-2-7b-chat-hf"
llama_model = AutoModelForCausalLM.from_pretrained(llama_model_id, torch_dtype=torch.float16, device_map="auto")
llama_tokenizer = AutoTokenizer.from_pretrained(llama_model_id)

# **Step 5:Text Generation Pipeline**
Initialize Text Generation Pipeline
Setting up a text generation pipeline using the Llama model for generating responses.
`max_length` is set to 4096 to handle longer conversations, `do_sample=True` allows for sampling when generating text,
and `temperature` and `top_p` control the randomness and diversity of the generated responses.

In [ ]:
response_generator = pipeline(
    "text-generation", model=llama_model, tokenizer=llama_tokenizer,
    torch_dtype=torch.float16, device_map="auto",
    max_length=4096, do_sample=True, temperature=0.6, top_p=0.95
)


# **Step 6:Generate Bot's Response**
 Define Function to Generate Response
The function `generate_response` generates a response based on the user input. The maximum response length is dynamically
calculated based on the length of the user's input to ensure appropriate response lengths.

In [ ]:
def generate_response(user_input):
    input_len = len(user_input.split())
    max_len = min(4096, input_len * 10 )
    return response_generator(user_input, max_length=max_len, do_sample=True)[0]['generated_text'].strip()

# **Step 7: Analyze Sentiment of User Input**
Define Function for Sentiment Analysis
This function analyzes the sentiment of the user's input using the sentiment analysis pipeline.
Based on the sentiment (positive or negative) and its confidence score, it returns a relevant response.
If the sentiment is very negative, the chatbot responds with an empathetic message.
If positive, it encourages the user, and if neutral or low confidence, it gives a general supportive message

In [ ]:
def analyze_sentiment(user_input):
    sentiment = sentiment_analyzer(user_input)[0]
    label, score = sentiment['label'], sentiment['score']
    if label == "NEGATIVE":
        return "Sorry you're feeling down. I'm here for you." if score > 0.9 else "It seems rough. I'm listening."
    elif label == "POSITIVE":
        return "Great to hear! Keep up the good vibes!" if score > 0.9 else "Glad you're feeling better."
    else:
        return "I'm here for you no matter what."

# **Step 8:Clean Redundant Phrases from Generated Text**
 Define Function to Clean Up Redundant Phrases
After generating a response, this function checks for redundant or overused phrases and removes them
to keep the conversation fresh and avoid repetitive replies.

In [ ]:
def clean_redundant(user_input, generated_text):
    phrases_to_remove = ["It seems rough.", "Take a deep breath."]
    for phrase in phrases_to_remove:
        generated_text = generated_text.replace(phrase, "").strip()

    return generated_text

# **Step 9:Main Chatbot Reply Logic**
Define Function for Chatbot Reply
This function combines sentiment analysis with text generation to form a chatbot response.
First, it analyzes the sentiment of the user input, generates a response based on that input,
and then cleans up any redundant phrases before returning the final chatbot response.

In [ ]:
def chatbot_reply(user_input):
    sentiment_msg = analyze_sentiment(user_input)
    generated_text = generate_response(user_input)
    cleaned_response = clean_redundant(user_input, generated_text)
    return f"{sentiment_msg} {cleaned_response}".strip()

# **Step 10:Track Chat History and Handle "Clear" Command**
 Initialize Chat History
This list stores the conversation history between the user and the chatbot. It is updated with every new user input.

In [ ]:
chat_history = []

# **Step 11:Track Chat History and Handle 'Clear' Command**
Define Function for Chat Interface
This function manages the interface by interacting with the user input.
It handles special commands like 'clear' to reset the conversation history, generates the chatbot's reply,
and appends the conversation to the chat history

In [ ]:
def interface_function(user_input):
    global chat_history
    if user_input.lower() == "clear":
        chat_history = []
        return "Chat history cleared."

    reply = chatbot_reply(user_input)
    chat_history.append(f"USER: {user_input}\nBOT: {reply}")
    return "\n\n".join(chat_history)

# **Step 12:Launch Gradio Interface**
 Set Up Gradio Interface
Gradio is used to create a simple and interactive web interface for the chatbot. The interface consists of
a text input for the user to type their messages, and it displays the chatbot's replies based on the sentiment analysis.

In [ ]:
interface = gr.Interface(
    fn=interface_function,
    inputs="text",
    outputs="text",
    title="Wellness Bot",
    description="Chat about your feelings, and the bot will respond based on sentiment. Type 'clear' to clear chat.",
    allow_flagging="never"
)

# **Step 13: Launch Gradio Interface**
Launch the Interface
This launches the Gradio interface, allowing users to interact with the chatbot in a web-based environment.
The `debug=True` flag helps in identifying issues if the code encounters any errors during execution.

In [ ]:
interface.launch(debug=True)